In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import os

TensorFlow 2.x selected.


In [0]:
# Load a toy dataset for the sake of this example
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data (these are Numpy arrays)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

val_size = 12000
train_size = len(x_train) - val_size

# Reserve 12,000 samples for validation
x_val = x_train[-val_size:]
y_val = y_train[-val_size:]
x_train = x_train[:-val_size]
y_train = y_train[:-val_size]

In [0]:
def build_model():
  activation_fn = tf.nn.elu
  regularization_fn = tf.keras.regularizers.l2
  #n_filters = 256
  #n_kernals = 3
  regularization_rate = 1e-4

  inputs = tf.keras.Input(shape=(28, 28, 1), name='original_img')
  x = tf.keras.layers.Conv2D(32, 3, activation=activation_fn, kernel_regularizer=regularization_fn(regularization_rate))(inputs)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(128, activation=activation_fn, kernel_regularizer=regularization_fn(regularization_rate))(x)
  predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

  model = tf.keras.Model(inputs, predictions, name='mnist_cnn')
  return model

In [0]:
tf.random.set_seed(0)

model_name = "minst_model"
try:
  model = tf.keras.models.load_model(model_name)
  print("Loaded model.")
except Exception:
  model = build_model()
  model.save(model_name)
  print("Built model.")

#model = build_model()
model.summary()

Loaded model.
Model: "mnist_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
original_img (InputLayer)    [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2769024   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________


In [0]:
lr_step = tf.Variable(0, trainable=False)
lr_boundaries = [400000, 600000]
lr_values = [1e-2, 1e-3, 1e-4]
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_boundaries, lr_values)
# Later, whenever we perform an optimization step, we pass in the step.
lr_schedule = learning_rate_fn(lr_step)

In [0]:
# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
val_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Prepare the metrics.
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Prepare the training dataset.
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [0]:
@tf.function
def train_step(x_batch_train, y_batch_train):
  with tf.GradientTape() as tape:
    logits = model(x_batch_train)
    loss_value = loss_fn(y_batch_train, logits)
    
    # Add extra losses created during this forward pass:
    loss_value += sum(model.losses)
  
  grads = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
  return logits, loss_value

In [0]:
#@tf.function
def validation_loop(val_dataset):
  # Run a validation loop at the end of each epoch.
  for x_batch_val, y_batch_val in val_dataset:
    val_logits = model(x_batch_val)
    # Update val metrics
    val_acc_metric(y_batch_val, val_logits)
  val_acc = val_acc_metric.result()
  val_acc_metric.reset_states()
  print('Validation acc: %s' % (float(val_acc),))

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

manager = tf.train.CheckpointManager(
    checkpoint, directory=checkpoint_dir, max_to_keep=5)

In [0]:
def train(train_dataset, epochs):
  # Iterate over epochs.
  #status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
  status = checkpoint.restore(manager.latest_checkpoint)

  print("Train on {} samples, validate on {} samples".format(val_size, train_size))
  for epoch in range(epochs):
    print('Epoch %d/%d' % (epoch + 1, epochs))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
      (logits, loss_value) = train_step(x_batch_train, y_batch_train)
      
      # Update training metric.
      train_acc_metric(y_batch_train, logits)

      # Log every 200 batches.
      if step % 100 == 0:
        print ('\r{}/{} - train_loss: {}'.format((step + 1) * batch_size, train_size, float(loss_value)), end='')
          #print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
          #print('Seen so far: %s samples' % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print ('\n{}/{} - train_loss: {} - train_acc: {}'.format((step + 1) * batch_size, train_size, float(loss_value), float(train_acc)))
    #print('Training acc over epoch: %s' % (float(train_acc),))
    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    validation_loop(val_dataset)

    # Save the model every 15 epochs
    if (epoch + 1) % 5 == 0:
      print("Saving model to %s" % (checkpoint_dir,))
      #checkpoint.save(file_prefix = checkpoint_prefix)
      manager.save()

In [0]:
train(train_dataset, epochs=5)

Train on 12000 samples, validate on 48000 samples
Epoch 1/5
44832/48000 - train_loss: 0.020668035373091698
48000/48000 - train_loss: 0.02058146707713604 - train_acc: 0.9999791383743286
Validation acc: 0.9807500243186951
Epoch 2/5
44832/48000 - train_loss: 0.020255522802472115
48000/48000 - train_loss: 0.020419897511601448 - train_acc: 1.0
Validation acc: 0.981333315372467
Epoch 3/5
44832/48000 - train_loss: 0.01914372481405735
48000/48000 - train_loss: 0.018081573769450188 - train_acc: 1.0
Validation acc: 0.981249988079071
Epoch 4/5
44832/48000 - train_loss: 0.01857433095574379
48000/48000 - train_loss: 0.017451060935854912 - train_acc: 1.0
Validation acc: 0.9815000295639038
Epoch 5/5
44832/48000 - train_loss: 0.016611458733677864
48000/48000 - train_loss: 0.01660006493330002 - train_acc: 0.9999791383743286
Validation acc: 0.9815833568572998
Saving model to ./training_checkpoints


In [0]:
#@tf.function
def evaluate(data, labels):
  predictions = model(data)
  t_loss = loss_fn(labels, predictions)

  test_loss = tf.keras.metrics.Mean(name='test_loss')
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
  test_loss(t_loss)
  test_accuracy(labels, predictions)
  return float(test_loss.result()), float(test_accuracy.result())

In [0]:
evaluate(x_test, y_test)

(0.07315120846033096, 0.982200026512146)